<img src="../images/cads-logo.png" style="height: 100px;" align=left>
<img src="../images/sklearn-logo.png" style="height: 100px;" align=right>

# Linear Regression

In this notebook you will learn what a linear regression is and for what it can be used.

We will start with a simple linear regression, which uses one predictive variable to predict the outcome, e.g. predicting a person's weight based only on his/her height. You will learn how to:
- fit a linear regression to data
- predict output values for new data
- measure the performance of the model

Then we will move on to a multidimensional linear regression, which uses multiple predictive variables to predict an outcome, e.g. predicting a person's weight based on both their height and their age.

Finally, we will discuss polynomial regression. For this, we will transform features into a higher dimensional space so that we can use a linear regression to capture non-linear relationships.

# Table of Contents
- [Linear Regression](#Linear-Regression)
- [Introduction to Linear Regression](#Introduction-to-Linear-Regression)
    - [Motivation](#Motivation)
- [Simple Linear Regression](#Simple-Linear-Regression)
    - [Learning Model Coefficients](#Learning-Model-Coefficients)
    - [How well does the model fit the data?](#How-well-does-the-model-fit-the-data?)
    - [Exercise - Create your first Linear Regression model](#Exercise---Create-your-first-Linear-Regression-model)
    - [Bonus Exercise - Implement your own error metrics](#Bonus-Exercise---Implement-your-own-error-metrics)
- [Multidimensional linear regression](#Multidimensional-linear-regression)
    - [Exercise - Effects of Advertising on Sales](#Exercise---Effects-of-Advertising-on-Sales)
- [Feature Scaling](#Feature-Scaling)
- [Handling Highly Correlated Features](#Handling-Highly-Correlated-Features)
- [Handling Categorical Predictors](#Handling-Categorical-Predictors)
- [Using Linear Regression for non-linear relations](#Using-Linear-Regression-for-non-linear-relations)
    - [Basis functions](#Basis-functions)
    - [Regression with polynomial basis functions](#Regression-with-polynomial-basis-functions)
    - [A comment on `PolynomialFeatures`](#A-comment-on-`PolynomialFeatures`)
- [Best Practice for Machine Learning](#Best-Practice-for-Machine-Learning)
    - [Exercise - Train and test the advertising dataset](#Exercise---Train-and-test-the-advertising-dataset)
- [Explore sklearns datasets](#Explore-sklearns-datasets)
- [Lasso, Ridge, and Elastic Net Regression: Regularization](#Lasso,-Ridge,-and-Elastic-Net-Regression:-Regularization)
- [Summary](#Summary)
- [Resources](#Resources)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import sklearn.preprocessing
import sklearn.pipeline
import sklearn.datasets

import warnings
warnings.filterwarnings("ignore")

sns.set()

## Introduction to Linear Regression
<img src ="../images/linear_regression.png">

### Motivation
Why linear regression?
- Easy to use (requires very little tuning)
- Low cost of computation
- Easy to interpret
- Basis for many other methods

## Simple Linear Regression

A simple linear regression predicts a **continuous response** using a **single, continuous feature** (also called "predictor" or "input variable"). It takes the following form:

$y = \beta_0 + \beta_1x$

What does each term represent?
- $y$ is the response
- $x$ is the feature
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for x, also called slope

<img src="../images/slope_intercept.png">

Together, $\beta_0$ and $\beta_1$ are called the **model coefficients**. Fitting a model to data means to "learn" the optimal values for these coefficients.

### Learning Model Coefficients

Generally speaking, coefficients are estimated using the **least squares criterion**, which means we find the line that minimizes the cumulative distance from the data. The most common metric for this distance is the **residual sum of squares (RSS)**, also known as the **sum of squared residuals (SSR)** or the **sum of squared errors (SSE)**, is the sum of the squares of residuals (deviations predicted from actual empirical value:

<img src="../images/estimating_coefficients.png">

What elements are present in the diagram?
- The black dots are the **observed values** of x and y.
- The blue line is the line that minimizes the sum of squared residuals, i.e. the **least squares line**.
- The red lines are the **residuals**, which are the distances between the observed values and the least squares line.

How do the model coefficients relate to the least squares line?
- $\beta_0$ is the **intercept** (the value of $y$ when $x$=0)
- $\beta_1$ is the **slope**, or angle, of the line (the change in $y$ divided by change in $x$)

Here is a graphical depiction of those calculations:

In [ ]:
# Create random data
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = -5 + 2 * x + rng.randn(50)
plt.scatter(x, y);

In [ ]:
x

In [ ]:
x.shape

We can use Scikit-Learn's ``LinearRegression`` estimator to fit this data and construct the best-fit line:

**Step 1 - Choose model hyperparameters**

Hyperparameters are model parameters that we select before fitting it to the data. These change the behaviour of our model. For a linear regression, the only hyperparameter we're interested in is whether or not to fit the intercept to our data. If we set this to `False`, then Python expects the data to be centered and simply sets $\beta_0 = 0$.

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
model

**Step 2 - Arrange data into a features matrix and target vector**

Scikit learn is an extremely flexible framework for machine learning. This flexibility comes at the cost of very strict requirements for features and target values.

- Features must be a 2D-array with the shape (number of samples, number of features), i.e. each row of X represents a data entry and each column represents a feature
- Target values must be a 1D-array with the shape (number of samples, )

In case we only have one feature, like here, we need to first transform the variable.

<img src="../images/numpy_array.png">

In [ ]:
print("Shape of x: {}".format(x.shape))
print(x[:5])

In [ ]:
print("Shape of y: {}".format(y.shape))
print(y[:5])

In [ ]:
x = x[:,np.newaxis]
print("Shape of x: {}".format(x.shape))
print(x[:5])

**Step 3 - Fit the model to your data**

In [ ]:
model.fit(x, y)

We can inspect the parameters $\beta_0, \beta_1$ of the model:

In [ ]:
# intercept, i.e. beta_0
model.intercept_

In [ ]:
# slope, i.e. beta_1
model.coef_

**Step 4 - Visually inspect the fit**

We can apply the model to our input features and visually inspect the fit.

In [ ]:
y_hat1=model.predict([[8.5]])
y_hat1

In [ ]:
y_hat2=model.intercept_+model.coef_[0]*8.5
y_hat2

In [ ]:
yfit = model.predict(x)

In [ ]:
print("Shape of yfit: {}".format(yfit.shape))
print(yfit)

In [ ]:
print(y)

The model applies the learned coefficients $\beta_0, \beta_1$ to each row of our feature matrix (a "row" here is simply a single value) and computes the output. We could do this manually as well to confirm that scikit-learn is doing what we expect it to, e.g.

In [ ]:
model.intercept_ + model.coef_ * x[0]

Let's plot the raw data and the predicted values using matplotlib

In [ ]:
plt.scatter(x, y);
plt.plot(x, yfit, color='red');

The slope and intercept of the data are contained in the model's fit parameters, which in Scikit-Learn are always marked by a trailing underscore. Here the relevant parameters are ``coef_`` and ``intercept_``:

In [ ]:
print("Model slope:    ", model.coef_)
print("Model intercept:", model.intercept_)

We see that the results are very close to the inputs, as we might hope. Can we measure how close, though?

### How well does the model fit the data?
The quality of a linear regression fit is most commonly described with the **mean squared error (MSE)**, which computes the average of the squared differences between observed and predicted target values:

$$MSE = \frac{1}{N} \cdot \sum_i^N (y_i - \hat y_i)^2$$

where:
- $y_i$ represents the observed target value for $x_i$
- $\hat y_i$ represents the predicted target value for $x_i$ as per the learned parameters.
- $N$ is equal to the number of observations

We can easily compute the mean squared error with scikit-learn;

In [ ]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_true=y, y_pred=yfit)
mse

In [ ]:
rmse=np.sqrt(mse)
rmse

In [ ]:
np.min(y),np.max(y)

An inherent shortcoming of this metric is that it depends on the scaling of the output variables, e.g. if our outputs and predicted variables are scaled by a factor of 2, then the MSE is scaled by a factor of 4. This doesn't mean that the model performs worse on larger target variables and better on smaller ones.

In [ ]:
mean_squared_error(y_true=2*y, y_pred=2*yfit)

To solve this, we introduce the **coefficient of determination** (also referred to as $R^2$ or R-Squared). $R^2$ describes the ratio of the mean squared error of the model compared to the mean squared error of the **null model**, which is simply a horizontal line through the mean of the observed target variables ($\beta_0 = mean(y_1, ..., y_N)$ and $\beta_1 = 0$)"

$$R^2 = 1 - \frac{MSE_{model}}{MSE_{null}}$$

where:
- $MSE_{model} = \frac{1}{N} \cdot \sum_i^N (y_i - \hat y_i)^2$ (as above)
- $MSE_{null} = \frac{1}{N} \cdot \sum_i^N (y_i - \bar y)^2$
- and $\bar y = mean(y_1, ..., y_N)$ is the mean of all observed target values.

In other words, $R^2$ is one minus the the ratio of the blue area to the red area.

<img src="../images/Coefficient_of_Determination.svg" />

Another way of phrasing this is that $R^2$ is the fraction of variance in the data that can be explained by the model. It (usually) lies between 0 and 1, and higher is better because it means that more variance is explained by the model. The remaining variance in data is due to complex relationships between features and target values not considered by the model and random noise.

Note that a perfect $R^2 = 1$ should be regarded with suspicion. This means that:

- The model fully describes the relationship between features and target values
- There is no randomness in the data

both of which are extremely unlikely in any real-life scenario.

Scikit-Learn also lets us compute the $R^2$ score:

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_true=y, y_pred=yfit)

Note that this score is robust towards value scaling:

In [ ]:
r2_score(y_true=2*y, y_pred=2*yfit)

*Note: $R^2$ can be negative in some situations. In particular, if no intercept was included in the fit, $R^2$ can become negative. In these cases, the learned model is worse than the null model.*

The $R^2$ necessarily increases for any predictor that's added to the data, even pure noise. The **Adjusted $R^2$** adjusts for this, and you should use this in place of the R-squared if comparing models with different numbers of predictors.

> ${R^2}_{Adj.} = 1- \frac {(1 - R^2)(N - 1)}{N - p - 1}$

> $N$ = size of data set <br>
> $p$ = number of predictors

In [ ]:
x.shape[1]

In [ ]:
def adj_r2(ytrue, ypred, N, p):
    return 1- ((1-r2_score(ytrue, ypred))*(N - 1))/(N - p - 1)

adj_r2(y, yfit, N=len(y),p=x.shape[1])

### Exercise - Create your first Linear Regression model

Given the following X and y, train a linear regression model and show the results.

- Create and fit the model using intercept=True
- Show the trained parameters (intercept and slope)
- Show the MSE and R2 for the predicted target values $\hat{y_i}$
- Plot the original datapoints and the regression line
- Repeat the previous steps with intercept=False

The data:

In [ ]:
rng = np.random.RandomState(1)
X = 100 * rng.rand(50)
y = 26 * X - 722 + rng.randn(50)*200

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
plt.scatter(X, y);

Create and fit the model

Hints:
- np.newaxis

In [ ]:
### Your code here

Show the trained parameters (intercept and slope)

Hints:
- Trained parameters have a trailing underscore
- Type "model." and then press TAB to have Jupyter show you all available methods and variables

In [ ]:
### Your code here

Show the MSE and R2 for the predicted target values $\hat{y_i}$

Hints:
- Metrics can be found in the module `sklearn.metrics`. Use TAB to view available metrics.

In [ ]:
### Your code here

Plot the original datapoints and the regression line

In [ ]:
### Your code here

Repeat the previous steps with intercept=False

In [ ]:
### Your code here

What does the intercept do?

## Multidimensional linear regression
The ``LinearRegression`` estimator is much more capable than this, however—in addition to simple straight-line fits, it can also handle multidimensional linear models of the form

$$y=\beta_0+\beta_1x_1+\beta_2x_2+⋯+\beta_nx_n$$

where there are multiple $x$ values. Geometrically, this is akin to fitting a plane to points in three dimensions, or fitting a hyper-plane to points in higher dimensions.

The multidimensional nature of such regressions makes them more difficult to visualize. Nonetheless, the general steps for fitting a model to data outlined for the one-dimensional case above remain the same.

In [ ]:
rng = np.random.RandomState(1)
X = 10 * rng.rand(100, 3)
# b0 = 0.5, b1 = 1.5, b2 = -2, b3 = 1
y = 0.5 + 1.5 * X[:, 0] - 2 * X[:, 1] + 1 * X[:, 2] + rng.randn(100)

model = LinearRegression()
model.fit(X, y)

print("Intercept:    {}".format(model.intercept_))
print("Coefficients: {}".format(model.coef_))

In [ ]:
yfit=model.predict(X)
mse=mean_squared_error(y,yfit)
r2=r2_score(y,yfit)
print(mse,r2)

Here the $y$ data is constructed from three random $x$ values, and the linear regression recovers the coefficients used to construct the data. In this way, we can use the single `LinearRegression` estimator to fit lines, planes, or hyperplanes to our data.

### Exercise - Effects of Advertising on Sales

Imagine that your company wants to increase sales on a certain product. You cannot increase sales directly, but you can adjust advertising.

The advertising dataset in the file `data/advertising.csv` contains information on money spent on advertising via various channels as well as revenue for a product. Each row represents a separate time interval, e.g. a week's worth of sales. For the sake of simplicity we'll ignore confounding factors like seasonality.

The features are:
- TV: advertising dollars spent on TV
- Radio: advertising dollars spent on Radio
- Newspaper: advertising dollars spent on Newspaper

and the response is:
- Sales: sales of a single product in a given market

All numbers are in thousands of dollars, e.g. 14.4 is equivalent to \$14,400.

In [ ]:
sales = pd.read_csv("../data/advertising.csv", index_col=0)
sales.head()

Explore the data and use simple, one-dimensional linear regression models to find answers to the following.
- What is the relation between each ad type and the sales?
- How well do the individual simple linear regression models describe this relation? (Hint: R2 score)

In [ ]:
### Your code here

In [ ]:
np.corrcoef(sales['TV'],sales['sales'])[0,1]**2

**Discussion questions:**

- Which ad types have the strongest and weakest influence on sales?
- For each ad type, how much is the expected sales given an additional budget of 10 thousand dollar for that ad type?

Let's see if we can improve the model performance with a multiple linear regression
- Train a multiple linear regression model on all ad types simultaneously. How well does it perform?
- How do the coefficients change versus the simple linear regression?

In [ ]:
### Your code here

- What are the expected sales given a new budget of TV=100, Radio=25 and Newspaper=25?

In [ ]:
### Your code here

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
print('10 units increase in TV channel, given radio and newspaper fixed, will increase the sale {:.3f}'.format(10*model.coef_[0]))

In [ ]:
print('10 units increase in radio channel, given TV and newspaper fixed, will increase the sale {:.3f}'.format(10*model.coef_[1]))

## Feature Scaling

For those machine learning algorithms which are based on the distance of the data points, the range (scale) of the features matter, because it defines the magnitude of the feature and those features in larger scales (e.g. Kilometers vs. Centimeters) will dominate the features in smaller scales. Therefore, as part of the pre-processing step in designing the predictive models, one needs to re-scale the features in the similar ranges.

#### Standardisation:

$$x_{scaled} = \frac{x - \bar {x}}{s}$$

Where $\bar{x} =  mean(x)$, and $s = std(x)$.

Therefore, $mean(x_{scaled}) = 0$ , and $std(x_{scaled}) = 1$.
In case of presence of outliers for the feature x, this method is not helpful to set the range of $x_{scaled}$.


#### Min-Max Scaling:

$$x_{scaled}=\frac{x-x_{min}}{x_{max} - x_{min}} $$

Therefore, $range(x) = [0 , 1]$


#### Mapping to a Gaussian distribution, PowerTransformer:

PowerTransformer aims to map data from any distribution to as close to a Gaussian (Normal) distribution as possible in order to stabilize variance and minimize skewness. In general, you'll only want to normalize your data if you're going to use a machine learning or statistics technique that assumes your data is normally distributed. Some examples of these include t-tests, ANOVAs, linear regression, and Gaussian naive Bayes.


### Feature Scaling in Python:

For each scaling methods "Standardisation" and Min-Max Scaling, we can call the relevant methods from `sklearn.preprocessing`.


#### Standardisation in Python:

`from sklearn.preprocessing import StandardScaler`

`scaler = StandardScaler().fit(x)`

`x_scaled = scaler.transform(x)`


#### Min-Max Scaling in Python:

`from sklearn.preprocessing import MinMaxScaler`

`scaler = MinMaxScaler().fit(x)`

`x_scaled = scaler.transform(x)`


#### Mapping to a Gaussian distribution, PowerTransformer:

`from sklearn.preprocessing import PowerTransformer`

`pt = PowerTransformer(method='box-cox', standardize=True)`

In PowerTransformer, you need to set the hyperparameter `method='box-cox'` if the data is positive. For negative data you need to set `method='yeo-johnson'`. Hyperparameter `standardize=True` applies zero-mean, unit-variance normalization to the transformed output.

`x_scaled = pt.fit_transform(x)`

**Example:**

In [ ]:
# set seed for reproducibility
np.random.seed(0)

from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer

# generate 1000 data points randomly drawn from an exponential distribution
original_data = np.random.exponential(size = 1000)[:,np.newaxis]

# StandardScaler scale the data between 0 and 1
scaler_std = StandardScaler().fit(original_data)
scaled_std = scaler_std.transform(original_data)

# MinMaxScaler scale the data between 0 and 1
scaler_Mm = MinMaxScaler().fit(original_data)
scaled_Mm = scaler_Mm.transform(original_data)

# MinMaxScaler scale the data between 0 and 1
scaler_PT = PowerTransformer().fit(original_data)
scaled_PT = scaler_PT.transform(original_data)

# plot both together to compare
fig, ax=plt.subplots(4,1)
fig.set_size_inches(10, 15)

sns.distplot(original_data, ax=ax[0])
ax[0].vlines(np.mean(original_data), ymin = 0, ymax = 1,
                               color = 'orange', label = 'mean')
ax[0].vlines(np.median(original_data), ymin = 0, ymax = 1,
                            color = 'red', label = 'median')
ax[0].vlines(np.mean(original_data) - np.std(original_data, ddof = 1), ymin = 0, ymax = 1,
                               color = 'green', label = '1-std below')
ax[0].vlines(np.mean(original_data) + np.std(original_data, ddof = 1), ymin = 0, ymax = 1,
                               color = 'purple', label = '1-std above')
ax[0].set_title("Original Data")


sns.distplot(scaled_std, ax=ax[1])
ax[1].set_title("StandardScaler: Scaled data")
ax[1].vlines(np.mean(scaled_std), ymin = 0, ymax = 1,
                               color = 'orange', label = 'mean')
ax[1].vlines(np.median(scaled_std), ymin = 0, ymax = 1,
                            color = 'red', label = 'median')
ax[1].vlines(np.mean(scaled_std) - np.std(scaled_std, ddof = 1), ymin = 0, ymax = 1,
                               color = 'green', label = '1-std below')
ax[1].vlines(np.mean(scaled_std) + np.std(scaled_std, ddof = 1), ymin = 0, ymax = 1,
                               color = 'purple', label = '1-std above')


sns.distplot(scaled_Mm, ax=ax[2])
ax[2].set_title("MinMaxScaler: Scaled data")
ax[2].vlines(np.mean(scaled_Mm), ymin = 0, ymax = 8,
                               color = 'orange', label = 'mean')
ax[2].vlines(np.median(scaled_Mm), ymin = 0, ymax = 8,
                            color = 'red', label = 'median')
ax[2].vlines(np.mean(scaled_Mm) - np.std(scaled_Mm, ddof = 1), ymin = 0, ymax = 8,
                               color = 'green', label = '1-std below')
ax[2].vlines(np.mean(scaled_Mm) + np.std(scaled_Mm, ddof = 1), ymin = 0, ymax = 8,
                               color = 'purple', label = '1-std above')


sns.distplot(scaled_PT, ax=ax[3])
ax[3].set_title("PowerTransformer: Scaled data")
ax[3].vlines(np.mean(scaled_PT), ymin = 0, ymax = .4,
                               color = 'orange', label = 'mean')
ax[3].vlines(np.median(scaled_PT), ymin = 0, ymax = .4,
                            color = 'red', label = 'median')
ax[3].vlines(np.mean(scaled_PT) - np.std(scaled_PT, ddof = 1), ymin = 0, ymax = .4,
                               color = 'green', label = '1-std below')
ax[3].vlines(np.mean(scaled_PT) + np.std(scaled_PT, ddof = 1), ymin = 0, ymax = .4,
                               color = 'purple', label = '1-std above')

plt.legend()
plt.show()

**Example: Impact of scaling on sales dataset**

In [ ]:
X = sales[["TV", "radio", "newspaper"]]
X.head()

In [ ]:
scaler = PowerTransformer().fit(X)
scaled_X = scaler.transform(X)
scaled_X[:4,:]

In [ ]:
# plot both together to compare
fig, ax=plt.subplots(1,2)
fig.set_size_inches(10, 8)

sns.distplot(X.iloc[1,:], ax=ax[0])
ax[0].vlines(np.mean(X.iloc[1,:]), ymin = 0, ymax = .3,
                               color = 'orange', label = 'mean')
ax[0].vlines(np.median(X.iloc[1,:]), ymin = 0, ymax = .3,
                            color = 'red', label = 'median')
ax[0].vlines(np.mean(X.iloc[1,:]) - np.std(X.iloc[1,:], ddof = 1), ymin = 0, ymax = .3,
                               color = 'green', label = '1-std below')
ax[0].vlines(np.mean(X.iloc[1,:]) + np.std(X.iloc[1,:], ddof = 1), ymin = 0, ymax = .3,
                               color = 'purple', label = '1-std above')
ax[0].set_title("X.TV")


sns.distplot(scaled_X[:,0], ax=ax[1])
ax[1].set_title("PowerTransform: X.TV")
ax[1].vlines(np.mean(scaled_X[:,0]), ymin = 0, ymax = .5,
                               color = 'orange', label = 'mean')
ax[1].vlines(np.median(scaled_X[:,0]), ymin = 0, ymax = .5,
                            color = 'red', label = 'median')
ax[1].vlines(np.mean(scaled_X[:,0]) - np.std(scaled_X[:,0], ddof = 1), ymin = 0, ymax = .5,
                               color = 'green', label = '1-std belo')
ax[1].vlines(np.mean(scaled_X[:,0]) + np.std(scaled_X[:,0], ddof = 1), ymin = 0, ymax = .5,
                               color = 'purple', label = '1-std above')

plt.legend()
plt.show()

In [ ]:
model = LinearRegression().fit(X, sales["sales"])

r2score = r2_score(
    sales["sales"], model.predict(X))
print("R2 of MLR:        {}".format(r2score))
print("MSE of MLR:        {}".format(mean_squared_error(sales["sales"], model.predict(X))))


print()
print("TV ad MLR:        {}".format(model.coef_[0]))
print()
print("Radio ad MLR:     {}".format(model.coef_[1]))
print()
print("Newspaper ad MLR: {}".format(model.coef_[2]))

In [ ]:
x_new=[[100,25,25]]
model.predict(x_new)

In [ ]:
model_scaled = LinearRegression().fit(scaled_X, sales["sales"])

r2score = r2_score(
    sales["sales"], model_scaled.predict(scaled_X))
print("R2 of MLR:        {}".format(r2score))
print("MSE of MLR:        {}".format(mean_squared_error(sales["sales"], model_scaled.predict(scaled_X))))


print()
print("TV ad MLR:        {}".format(model_scaled.coef_[0]))
print()
print("Radio ad MLR:     {}".format(model_scaled.coef_[1]))
print()
print("Newspaper ad MLR: {}".format(model_scaled.coef_[2]))

In [ ]:
model_scaled.predict(scaler.transform(x_new))

## Handling Highly Correlated Features

Highly correlated features (also known as multicollinearity) can lead to decreased generalization performance on the test set due to high variance and less model interpretability.

There are several ways to deal with multicollinearity problems in a model:

- Remove highly correlated predictors from the model
- Principal Components Analysis (PCA)
- Use regression models that cut the number of predictors to a smaller set of uncorrelated components (e.g Ridge Regression)

In [ ]:
size = 100

#A helper method for pretty-printing linear models
def pretty_print_linear(coefs, intercept, names = None, sort = False):
    if names == None:
        names = ["x%s" % x for x in range(1,len(coefs)+1)]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    st = " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)
    return " + ".join([str(round(intercept, 3)), st])

#We run the method 10 times with different random seeds
for i in range(10):
    print ("Random seed %s" % i)
    print()
    np.random.seed(seed=i)
    X_seed = np.random.normal(0, 1, size)
    X1 = X_seed + np.random.normal(0, .1, size)
    X2 = 2*X1+ np.random.normal(0, .1, size)
    X3 = X_seed + np.random.normal(0, .1, size)
    Y = X1 + X2 + X3 + np.random.normal(0, 1, size)
    X = np.array([X1, X2, X3]).T

    lr = LinearRegression()
    lr.fit(X,Y)
    print ("Linear model:", pretty_print_linear(lr.coef_, lr.intercept_))

## Handling Categorical Predictors

So far, all of our predictors have been numeric. What if one of our predictors was categorical?

Categorical predictors cannot be used in the regression equation just as they are. We need to transform them into dummy variables.

### Dummy variables

We need to represent all data numerically. If a predictor only has two categories, we can simply create a dummy variable that represents the categories as a binary value:

<img src='../images/Dummy1.png'>

<img src='../images/Dummy2.png'>

To create columns of dummy variables in our data frame, we can use `pd.get_dummies()`. Adding the `drop_first = True` argument adds $k-1$ columns to the DataFrame for a variable with $k$ categories.

### Example
For `insurance` data set, create a linear regression model to predict `charges` based on the rest of the given variabled.

In [ ]:
insurance = pd.read_csv('../data/insurance.csv')
insurance.head()

In [ ]:
insurance.info()

In [ ]:
insurance.describe()

In [ ]:
insurance.describe(include='object')

In [ ]:
sns.pairplot(insurance, hue='sex', corner=True, diag_kind="auto");

In [ ]:
sns.pairplot(insurance, hue='smoker', palette="husl", corner=True);

In [ ]:
sns.pairplot(insurance, hue='region', palette="deep");

In [ ]:
X=insurance.drop('charges', axis=1)
y=insurance.charges
X

In [ ]:
X_dum=pd.get_dummies(X, drop_first=True)
X_dum

In [ ]:
lm = LinearRegression().fit(X_dum,y)

r2score = r2_score(y, lm.predict(X_dum))
print("R2 = {:.2f}".format(r2score))

In [ ]:
lm.coef_

In [ ]:
print('y=({:.2f})+({:.2f}*age)+({:.2f}*bmi)+({:.2f}*children)+({:.2f}*sex_male)+({:.2f}*smoker_yes)+({:.2f}*region_northwest)+({:.2f}*region_southeast)+({:.2f}*region_southwest)'.\
      format(lm.intercept_,lm.coef_[0], lm.coef_[1],  lm.coef_[2],  lm.coef_[3],lm.coef_[4],lm.coef_[5],lm.coef_[6],lm.coef_[7] ))

How do we interpret the **smoker_yes coefficient**? For a given age, bmi, number of children, sex, and region; the amount of insurance charge for smokers is `23848.53` higher than a non smoker.

### Exercise:

How do we interpret the **sex_male coefficient**?

## Using Linear Regression for non-linear relations

Consider the following case.

In [ ]:
np.random.seed(42)

X = (10 * (2 * np.random.rand(100) - 1))[:, np.newaxis]
# b0 = 15, b1 = 0, b2 = -1.7, b3 = 0.2
y = 15 - 1.7 * X**2 + 0.2 * X**3 + np.random.normal(scale=5, size=X.shape)

plt.scatter(X, y, label = 'DataPoints')

In [ ]:
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

plt.scatter(X, y, label = 'DataPoints')
plt.plot(X, y_pred, c = 'red', label = 'Prediction Line')
plt.legend();

No matter how hard we try, we'll never be able to fit a line to capture the non-linear relationship between the features and the target value.

### Basis functions

A trick to adapt linear regression to nonlinear relationships between features and targets is to transform the data and generate new features from the existing ones using basis functions. A very common set of basis functions are **polynomial basis functions** $f_n(x) = x^n$, which transform a simple linear regression from

$$ y = \beta_0 + \beta_1 x $$

into

$$ y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + \ldots + \beta_n x^n$$

We are free to choose how many polynomials to include. In fact, the basis functions $f(x)$ can be practically anything; we are free to transform features however we deem necessary.

This can, of course, also be extended to multiple linear regression so that, e.g.

$$ y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 $$

could become

$$ y = \beta_0 + \beta_{1a} x_1 + \beta_{1b} x_1^2 + \beta_{2a} x_2 + \beta_{2b} x_2^2$$

Note that this is still a linear model. Linearity in the context of modelling means that the coefficients $\beta_i$ are only ever added to (or subtracted from) each other. The basis functions $f(x)$ may very well be non-linear, though. For example,

$$ y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2 $$

is a linear model, despite the $x_1 x_2$ term, whereas

$$ y = \beta_0 + \frac{\beta_1 x}{\beta_2 + x}$$

is a non-linear model, despite only having one feature, because the coefficients are divided by each other.

**All we are doing is engineering new features to capture nonlinear patterns!**

### Regression with polynomial basis functions

The polynomial basis functions are so common and useful that this transformation is built into Scikit-Learn.

In [ ]:
x_1d = np.array([2, 3, 4])[:, np.newaxis]
x_1d

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly.fit(x_1d)
x_poly = poly.transform(x_1d)
x_poly

We see here that the transformer has added columns to our array corresponding to $x^n$.

`include_bias` adds an $x^0 = 1$ column as well. This can be useful for more advanced models but in the case of a linear regression, this has the same effect as adding an intercept term during the fit itself.

In [ ]:
poly = PolynomialFeatures(degree=4, include_bias=True)
poly.fit(x_1d)
x_poly = poly.transform(x_1d)
x_poly

The `fit()` and `transform()` syntax may seem cumbersome but plays into Scikit-Learn's strength as a flexible framework. All of the preprocessors/transformers that are part of the [Pipeline API](https://scikit-learn.org/stable/modules/compose.html#pipeline) as well as the actual machine learning models implement these two functions. That means we can use [pipelines](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) to chain together many different steps into a single object. This ensures that our code stays concise and becomes reusable. A pipeline will apply the `fit()` and `transform()` functions of each individual step and pass along the resulting data to the next step.

For example, we can construct a pipeline that applies a polynomial transformation to the $3^{rd}$ degree, scales the resulting features to lie between 0 and 1, and then trains a linear regression on the resulting features.

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
poly_pipeline = make_pipeline(
    PolynomialFeatures(degree=3, include_bias=False),
    MinMaxScaler(),
    LinearRegression())
poly_pipeline

In [ ]:
poly_pipeline.steps

We can use the pipeline as a single object to fit the non-linear relationship from above.

In [ ]:
poly_pipeline.fit(X, y)
X_pred = np.linspace(X.min(), X.max(), 100)[:, np.newaxis]
y_pred = poly_pipeline.predict(X_pred)
plt.scatter(X, y, label = 'DataPoints')
plt.plot(X_pred, y_pred, c = 'red', label = 'Prediction Curve')
plt.legend();

In [ ]:
r2_score(y,poly_pipeline.predict(X) )

Our polynomial expansion accurately captures the non-linear relationship!

### A comment on `PolynomialFeatures`
The `PolynomialFeatures` preprocessor doesn't just apply exponents to features but also looks at interaction terms $x_i^n x_j^m$. For example, applying `PolynomialFeatures(degree=2)` to the following linear model:

$$ y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 $$

will result in:

$$ y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1^2 + \beta_4 x_2^2 + \beta_5 x_1 x_2$$

`PolynomialFeatures` will generate all interaction terms where the sum of the exponents is less than or equal to `degree`. So applying `PolynomialFeatures(degree=3)` to the linear model above results in:

$$
\begin{aligned}
y = \beta_0 &+ \beta_1 x_1 + \beta_2 x_2\\
            &+ \beta_3 x_1^2 + \beta_4 x_2^2\\
            &+ \beta_5 x_1 x_2\\
            &+ \beta_6 x_1^3 + \beta_7 x_2^3\\
            &+ \beta_7 x_1^2 x_2 + \beta_8 x_2^2 x_1\\
\end{aligned}
$$

Keep this in mind when generating polynomial expansions: the number of resulting features can become computationally problematic if `degree` is too large. For example, if we start with 10 features, a polynomial expansion with `degree=4` will result in 1000 features:

In [ ]:
np.random.seed(42)

x_1d = np.random.randint(low=-5, high=5, size=(5, 10))
x_1d

In [ ]:
x_poly = PolynomialFeatures(degree=4).fit_transform(x_1d)
print("Original shape:    {}".format(x_1d.shape))
print("Transformed shape: {}".format(x_poly.shape))

In [ ]:
# set interaction_only = True only creates x_i * x_j for i != j
x_poly_inter = PolynomialFeatures(degree=4, interaction_only = True).fit_transform(x_1d)
print("Interaction Only Transformed shape: {}".format(x_poly_inter.shape))

## Best Practice for Machine Learning
In the previous examples and exercises, we have assessed the performance of the trained model on the data it was trained on. This is problematic, as we have no guarantee that the model will capture the true relationship between the features and target values. Consider the following example:

<center><img src="../images/overfitting.png" /></center>

The black dots represent the raw input data and the two lines represent two models trained on this data.

- The blue line perfectly predicts the target value for each data point. This model has a perfect performance, i.e. $R^2 = 1$, when assessed on the training data. However, it clearly does not capture the true relationship between the feature and the target value.
- The black line, although not a perfect fit, i.e. $R^2 < 1$, much more accurately describes the true relationship between feature and target.

To avoid this phenomenon, called **overfitting**, we can split our data into a training and a test data set. This allows us to train the model on one part of the data and then assess its performance on data it has never seen to determine how well it generalizes to new data.

We can use the `sklearn.model_selection.train_test_split` function to divide the data into 2 sets for us.

In [ ]:
from sklearn.model_selection import train_test_split

# Generate random data
rng = np.random.RandomState(42)
X = 10 * rng.rand(100, 3)
y = 0.5 + np.dot(X, [1.5, -2., 1.]) + rng.randn(100)

In [ ]:
np.dot([1,2,1], [1,-1,0])==(1*1+2*(-1)+1*0)

In [ ]:
print("X.shape", X.shape)
print("y.shape", y.shape)

Now we divide the data into train and test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25, random_state=42)

In [ ]:
print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape)
print("y_test.shape", y_test.shape)

We now have a dataset of 75 entries that we can train the model on and a test dataset of 25 entries that we can use to assess the performance of the model.

In [ ]:
model = LinearRegression().fit(X_train, y_train)
print("Intercept: {}".format(model.intercept_))
print("Slope:     {}".format(model.coef_))

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
print("Train R2 Score: {}".format(r2_train))
print("Test R2 Score:  {}".format(r2_test))

The performance on the training data will typically be better than the test score. In general, we want to make sure that training and test performance are similar and maximal. We will go into more detail about tuning models, overfitting, and assessing their performance at a later point in this course. For now, keep in mind that we should separate our data into a training and a test set.

There is no ironclad rule about how large these two subsets of our data should be. In general, more training data will result in a model that better captures the true relationship between features and target.

### Exercise - Train and test the advertising dataset

Train a multiple linear regression on the advertising dataset as above, but this time reserve a fraction of the data as a test set to assess the performance. Do this with the following splits:
- Training: 50%, Test: 50%
- Training 95%, Test: 5%
- Training 5%, Test: 95%

Hint: The `test_size` argument of `train_test_split` takes a number between 0 and 1 indicating the relative size of the test set, e.g. 0.3 corresponds to "Reserve 30% of the data as a test set"

In [ ]:
### Your code here

X = sales[["TV", "radio", "newspaper"]]
y = sales["sales"]

We can use the following rules of thumb regarding the train and test performances:
- If $R^2_{train} \approx R^2_{test}$ and the performance is good, then our model is optimally trained.
- If $R^2_{train} \approx R^2_{test}$ but the performance is poor
    - We need more training data or
    - The algorithm doesn't accurately capture the relationship between features and target value, e.g. trying to perform a linear regression on non-linear data.
- If $R^2_{train} \gg R^2_{test}$ then the model is most likely overfitting on the data. We will talk about solutions to this problem soon.
- If $R^2_{train} \ll R^2_{test}$ then we most likely do not have sufficient test data to correctly assess the performance. More robust performance assessments, like **cross-validation** will solve this problem for us.

## Explore sklearns datasets

Sklearn provides both toy as well as real-world datasets: https://scikit-learn.org/stable/datasets/index.html.

We can load these with the built-in `sklearn.datasets.load_*()` or `sklearn.datasets.fetch_*()` functions. Python will download these datasets if they are not already saved locally.

In [ ]:
diabetes = sklearn.datasets.load_diabetes()

In [ ]:
diabetes

The resulting object is a dictionary with various entries, such as the data description, the features, feature names, etc.

In [ ]:
diabetes.keys()

In [ ]:
print(diabetes.DESCR)

In [ ]:
diabetes.feature_names

In [ ]:
diabetes.target

**Exercise - Boston Housing Data**

We're going to predict housing prices

1. Load the sklearn dataset of the Boston house prices.
2. Use a multiple linear regression to predict housing prices
    - Divide the data into a training and test data set (70% training/30% test split)
    - Train a multiple linear regression model
    - Assess the performance of the model using the $R^2$ score
    - Play around with the train/test split size to see how the fit changes

In [ ]:
### Your code here

In [ ]:
boston = sklearn.datasets.load_boston()
print("Keys in Boston dataset: {}".format(boston.keys()))
print(boston.DESCR)

In [ ]:
### Your code here
X = boston.data
y = boston.target

3. Try to improve the regression by using polynomial features
    - Create an sklearn pipeline that generates polynomial features and then trains a multiple linear regression on these features
    - Assess the performance ($R^2$ score) of this polynomial regression on the training and test data
    - Do this for polynomial degrees 2, 3, and 4.
    - Compare your results with the multiple linear regression above. What are your observations?

In [ ]:
### Your code here

# Lasso, Ridge, and Elastic Net Regression: Regularization

Welcome to Regularization!

With the higher degree polynomials we have seen a situation were there are a lot of features and the model tends to overfit.

Regularization is putting a penalty on the size of the coefficients to prevent overfitting.

In this Notebook we will go into Ridge and Lasso Regression, how they prevent overfitting and how they can deal with a large number of features.

## Lasso, Ridge, and Elastic Net

**Overview**
Ridge and Lasso regression are powerful techniques generally used for creating parsimonious models in presence of a ‘large’ number of features. Here ‘large’ can typically mean either of two things:

- Large enough to enhance the tendency of a model to overfit (as low as 10 variables might cause overfitting)
- Large enough to cause computational challenges. With modern systems, this situation might arise in case of millions or billions of features

Though Ridge and Lasso might appear to work towards a common goal, the inherent properties and practical use cases differ substantially. Generally, they work by penalizing the magnitude of coefficients of features along with minimizing the error between predicted and actual observations. These are called ‘regularization’ techniques. Regularization reduces the model complexity and prevents over-fitting. The key difference is in how they assign penalty to the coefficients:

**Regression:**

$$\hat{y} = \beta_0 + \beta_1  x_1 + \beta_2  x_2 + ... + \beta_n  x_n $$

- Minimize Error: $$\sum_{i=1}^{m} {(y - \hat{y})^2} = \sum_{i=1}^{m} {(y -  \beta_0 - \beta_1  x_1 - \beta_2  x_2 - ... - \beta_n  x_n )^2}$$

**Lasso Regression:**

**LASSO** stands for ``Least Absolute Shrinkage and Selection Operator`` where emphasis on the 2 key words – ‘absolute‘ and ‘selection‘.

Lasso regression performs **L1 regularization**, i.e. it adds a factor of sum of absolute value of coefficients in the optimization objective.

- Minimize Error: $$\sum_{i=1}^{m} {(y - \hat{y})^2} + \alpha\sum_{i=1}^{n} {|\beta_i|} = \sum_{i=1}^{m} {(y -  \beta_0 - \beta_1  x_1 - \beta_2  x_2 - ... - \beta_n  x_n )^2} + \alpha\sum_{i=1}^{n} {|\beta_i|}$$

Thus, lasso regression optimizes the following:

**Objective = RSS + α * (sum of absolute value of coefficients)**

Here, α (alpha) works similar to that of ridge and provides a trade-off between balancing RSS and magnitude of coefficients. Like that of ridge, α can take various values. Lets iterate it here briefly:

1. α = 0: Same coefficients as simple linear regression
2. α = ∞: All coefficients zero (same logic as before)
3. 0 < α < ∞: coefficients between 0 and that of simple linear regression

**Ridge Regression:**


As mentioned before, ``ridge regression`` performs ‘L2 regularization‘, i.e. it adds a factor of sum of squares of coefficients in the optimization objective.

- Minimize Error: $$\sum_{i=1}^{m} {(y - \hat{y})^2}+ \alpha \sum_{i=1}^{n} {\beta_i^2} = \sum_{i=1}^{m} {(y -  \beta_0 - \beta_1  x_1 - \beta_2  x_2 - ... - \beta_n  x_n )^2} + \alpha \sum_{i=1}^{n} {\beta_i^2}$$

Thus, ridge regression optimizes the following:

**Objective = RSS + α * (sum of square of coefficients)**

Here, α (alpha) is the parameter which balances the amount of emphasis given to minimizing RSS vs minimizing sum of square of coefficients. α can take various values:

1. α = 0:
    - The objective becomes same as simple linear regression.
    - We’ll get the same coefficients as simple linear regression.
2. α = ∞:
    - The coefficients will be zero. Why? Because of infinite weightage on square of coefficients, anything less than zero will make the objective infinite.
3. 0 < α < ∞:
    - The magnitude of α will decide the weightage given to different parts of objective.
    - The coefficients will be somewhere between 0 and ones for simple linear regression.

**Elastic Net Regression:**

- Minimize Error: $$\sum_{i=1}^{m} {(y - \hat{y})^2}+ \lambda_1 \sum_{i=1}^{n} {|\beta_i|} + \lambda_2 \sum_{i=1}^{n} {\beta_i^2} = \sum_{i=1}^{m} {(y -  \beta_0 - \beta_1  x_1 - \beta_2  x_2 - ... - \beta_n  x_n )^2} + \lambda_1 \sum_{i=1}^{n} {|\beta_i|} + \lambda_2 \sum_{i=1}^{n} {\beta_i^2}$$

In `sklearn`, the relationship between $\lambda_1$ and $\lambda_2$ is defined by two parameters in `ElasticNet` function `alpha` and `l1_ratio` where:
$$\alpha = \lambda_1 + \lambda_2$$
and $$ l1-ratio = \frac {\lambda_1}{(\lambda_1 + \lambda_2)}$$

For example, if $\alpha = 1$, and l1_ratio = 0.3, then:
$$ {\lambda_1 + \lambda_2 = 1} \\ { \frac {\lambda_1}{(\lambda_1 + \lambda_2)} = 0.3}$$
Therefore:
$$ {\lambda_1 = 0.3} \\  {\lambda_2 = 0.7}$$

In [ ]:
# x1=x2
# y=0.5+x1+x2 ------> [0.5, 1, 1]

print(0.5**2+1+1)

#y=0.5 + 2*x1 ------> [0.5, 2, 0]
print(0.5**2+2**2)

#y=0.5 + 2*x2 ------> [0.5, 0, 2]
print(0.5**2+2**2)


 **Example - Lasso on Boston with Polynomials**

- Load again the boston dataset from sklearn.
- Divide the data into 30% test and 70% train set.
- Fit a Polynomial Lasso Regression on the train data
    - Use degree=2, alpha=0.1, max_iter=100000
- What is the R2 for train and test? How many features were selected?
- Now try:
    - change PolynomialLasso to set interaction_only=True in PolynomialFeatures
    - degree=3, alpha=1, max_iter=100000

In [ ]:
from sklearn.linear_model import Lasso

def PolynomialLasso(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         Lasso(**kwargs))

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()
X = boston.data
y = boston.target

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X1, X2, y1, y2 = train_test_split(X, y,random_state=42,test_size=0.3)

In [ ]:
Lasso_Poly2_boston = PolynomialLasso(2, alpha = 0.1, max_iter=1e5)
Lasso_Poly2_boston.fit(X1, y1)

In [ ]:
print("Train score", Lasso_Poly2_boston.score(X1, y1))
print("Test score", Lasso_Poly2_boston.score(X2,y2))

In [ ]:
Lasso_Poly2_boston.steps[1][1].coef_

In [ ]:
k = Lasso_Poly2_boston.steps[1][1].coef_
print("Features all", len(k))
print("Features used", sum(k != 0))
print("Features NOT used", sum(k == 0))

In [ ]:
Lasso_Poly3_boston = PolynomialLasso(3, alpha = 1, max_iter=1e5)
Lasso_Poly3_boston.fit(X1, y1)

print("Train score", Lasso_Poly3_boston.score(X1, y1))
print("Test score", Lasso_Poly3_boston.score(X2,y2))
k = Lasso_Poly3_boston.steps[1][1].coef_
print("Features all", len(k))
print("Features used", sum(k != 0))
print("Features NOT used", sum(k == 0))

In [ ]:
def PolynomialLasso_inter(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree,
                                            interaction_only=True),
                         Lasso(**kwargs))

In [ ]:
Lasso_Poly3_inter_boston = PolynomialLasso_inter(3,
                                                 alpha = 1,
                                                 max_iter=1e5)
Lasso_Poly3_inter_boston.fit(X1, y1)

print("Train score", Lasso_Poly3_inter_boston.score(X1, y1))
print("Test score", Lasso_Poly3_inter_boston.score(X2,y2))
k = Lasso_Poly3_inter_boston.steps[1][1].coef_
print("Features all", len(k))
print("Features used", sum(k != 0))
print("Features NOT used", sum(k == 0))

**Exercise: Ridge & ElasticNet on Boston with Polynomials**
- Fit a Polynomial Ridge Regression on the train data
    - Use degree=2, alpha=0.1, max_iter=100000
- What is the R2 for train and test? How many features were selected?
- Now try:
    - change PolynomialRidge to set interaction_only=True in PolynomialFeatures
    - degree=3, alpha=1, max_iter=100000
    
- Fit a Polynomial Elastic Net Regression on the train data
    - Use degree=2, alpha=0.1, max_iter=100000
- What is the R2 for train and test? How many features were selected?
- Now try:
    - change PolynomialElastic to set interaction_only=True in PolynomialFeatures
    - degree=3, alpha = 1, l1_ratio=0.5, max_iter=100000

In [ ]:
from sklearn.linear_model import Ridge

def PolynomialRidge(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         Ridge(**kwargs))

In [ ]:
def PolynomialElastic_inter(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree, interaction_only=True),
                         ElasticNet(**kwargs))

In [ ]:
Elastic_Poly3_inter_boston = PolynomialElastic_inter(3, alpha = 1,
                                                     l1_ratio=0.7,
                                                     max_iter=1e5)
Elastic_Poly3_inter_boston.fit(X1, y1)
print("Train score", Elastic_Poly3_inter_boston.score(X1, y1))
print("Test score", Elastic_Poly3_inter_boston.score(X2,y2))
k = Elastic_Poly3_inter_boston.steps[1][1].coef_
print("Features all", len(k))
print("Features used", sum(Elastic_Poly3_inter_boston.steps[1][1].coef_ != 0))
print("Features NOT used", sum(Elastic_Poly3_inter_boston.steps[1][1].coef_ == 0))

## Summary

Now that we have a fair idea of how ridge and lasso regression work, lets try to consolidate our understanding by comparing them and try to appreciate their specific use cases. Lets analyze these under three buckets:

1. Key Difference

    - **Ridge:** It includes all (or none) of the features in the model. Thus, the major advantage of ridge regression is *coefficient shrinkage* and *reducing model complexity*.
    - **Lasso:** Along with *shrinking coefficients*, lasso performs *feature selection* as well. (Remember the ‘selection‘ in the lasso full-form?) As we observed earlier, some of the coefficients become exactly zero, which is equivalent to the particular feature being excluded from the model.
    
With advancements in Machine Learning, ridge and lasso regression provide very good solutions as they give much **better output**, require **fewer tuning parameters** and can be **automated** to a large extend.

2. Typical Use Cases

    - **Ridge:** It is majorly used to prevent *overfitting*. Since it includes all the features, it is not very useful in case of exorbitantly high #features, say in millions, as it will pose *computational challenges*.
    - **Lasso:** Since it provides *sparse solutions*, it is generally the model of choice (or some variant of this concept) for modelling cases where the #features are in millions or more. In such a case, getting a sparse solution is of great computational advantage as the features with zero coefficients can simply be ignored.
    
3. Presence of Highly Correlated Features

    - **Ridge:** It generally works well even in presence of highly correlated features as it will include all of them in the model but the *coefficients will be distributed among them depending on the correlation*.
    - **Lasso:** It arbitrarily selects any one feature among the highly correlated ones and reduced the coefficients of the rest to zero. Also, the chosen variable changes randomly with change in model parameters. This generally *doesn’t work that well as compared to ridge regression*.
    
Along with Ridge and Lasso, ``Elastic Net`` is another useful techniques which combines both ``L1`` and ``L2`` regularization. It can be used to balance out the pros and cons of Ridge and Lasso regression.

In [ ]:
size = 100

#We run the method 10 times with different random seeds
for i in range(10):
    print ("Random seed %s" % i)
    print()
    np.random.seed(seed=i)
    X_seed = np.random.normal(0, 1, size)
    X1 = X_seed + np.random.normal(0, .1, size)
    X2 = 2*X1+ np.random.normal(0, .1, size)
    X3 = X_seed + np.random.normal(0, .1, size)
    Y = X1 + X2 + X3 + np.random.normal(0, 1, size)
    X = np.array([X1, X2, X3]).T


    lr = LinearRegression()
    lr.fit(X,Y)
    print ("Linear model:", pretty_print_linear(lr.coef_, lr.intercept_))
    ridge = Ridge(alpha=10)
    ridge.fit(X,Y)
    print ("Ridge model:", pretty_print_linear(ridge.coef_, ridge.intercept_))
    print()

## Resources

- To go much more in-depth on linear regression, read Chapter 3 of [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/), from which this lesson was adapted. Alternatively, watch the [related videos](http://www.dataschool.io/15-hours-of-expert-machine-learning-videos/) or read my [quick reference guide](http://www.dataschool.io/applying-and-interpreting-linear-regression/) to the key points in that chapter.
- To learn more about Statsmodels and how to interpret the output, DataRobot has some decent posts on [simple linear regression](http://www.datarobot.com/blog/ordinary-least-squares-in-python/) and [multiple linear regression](http://www.datarobot.com/blog/multiple-regression-using-statsmodels/).
- This [introduction to linear regression](http://people.duke.edu/~rnau/regintro.htm) is much more detailed and mathematically thorough, and includes lots of good advice.
- This is a relatively quick post on the [assumptions of linear regression](http://pareonline.net/getvn.asp?n=2&v=8).
- Feature Selection: (https://www.datacamp.com/community/tutorials/feature-selection-python)
- Feature Engineering: (https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b)
- Handling Highly Correlated Features: (https://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/)